In [ ]:
#| default_exp utils

# Utils

> App setup and utility functions

In [ ]:
#| export
from fastcore.utils import *
from fasthtml.common import *
import fasthtml.common as fh
from fasthtml.jupyter import *
from fastlucide.icons import spritesheet
from fastlucide.core import _style_str, sz_attrs
from enum import Enum
from fastcore.meta import delegates
import re

In [ ]:
#| exports
# nbdev workaround to expose spritesheet in `__all__`
spritesheet = spritesheet

### Basecoat headers

In [ ]:
#| export
# Additional script needed for sliders
slider_script = Script("""
const updateSlider = (el) => {
    const min = parseFloat(el.min || 0);
    const max = parseFloat(el.max || 100);
    const value = parseFloat(el.value);
    const percent = (max === min) ? 0 : ((value - min) / (max - min)) * 100;
    el.style.setProperty('--slider-value', `${percent}%`);
};
""")
# For some reason the text tailwind classes are not being properly generated
text_css = Style("""
.text-muted-foreground { color: var(--muted-foreground); }
.text-foreground { color: var(--foreground); }
.hover\\:text-foreground:hover { color: var(--foreground); }
.bg-accent { background-color: var(--accent); }
.bg-border { background-color: var(--border); }
.bg-card { background-color: var(--card); }
.bg-muted { background-color: var(--muted); }
.bg-primary { background-color: var(--primary); }
.bg-background { background-color: var(--background); }
.border-border { border-color: var(--border); }
""")
theme_script = Script("""
(() => {
  const stored = localStorage.getItem('themeMode');
  const dark = stored ? stored === 'dark' : matchMedia('(prefers-color-scheme: dark)').matches;
  if (dark) document.documentElement.classList.add('dark');
  
  document.addEventListener('basecoat:theme', (e) => {
    const mode = e.detail?.mode || (document.documentElement.classList.contains('dark') ? 'light' : 'dark');
    const isDark = mode === 'dark';
    document.documentElement.classList.toggle('dark', isDark);
    localStorage.setItem('themeMode', isDark ? 'dark' : 'light');
  });
})();
""")

deps = {
    'scripts': [
        'https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4',
        'https://cdn.jsdelivr.net/npm/basecoat-css@0.3.6/dist/js/all.min.js'
    ],
    'links': [
        'https://cdn.jsdelivr.net/npm/basecoat-css@0.3.6/dist/basecoat.cdn.min.css'
    ]
}



In [ ]:
#| export
def get_static_headers():
    hdrs = []
    for p in Path("./static").glob("*"):
        if p.suffix == ".js":    hdrs.append(Script(src=f"/static/{p.name}"))
        elif p.suffix == ".css": hdrs.append(Link(rel="stylesheet", href=f"/static/{p.name}"))
    return tuple(hdrs)

In [ ]:
#| export
def get_cdn_headers():
    scripts = tuple(Script(src=url) for url in deps['scripts'])
    links = tuple(Link(rel='stylesheet', href=url) for url in deps['links'])
    return scripts + links

def make_hdrs(offline=False):
    hdrs = get_static_headers() if offline else get_cdn_headers()
    return (theme_script,) + hdrs + (text_css, slider_script)

In [ ]:
#| export
def basecamp_icons():
    return [
        "chevron-down",
        "chevron-right",
        "chevron-left",
        "circle-alert",
        "arrow-right",
        "arrow-left",
        "ellipsis",
        "loader-circle",
        "sun",
        "moon",
        "circle-check"
    ]


In [ ]:
#| export
def code_highlight_headers():
    return (
        Link(rel="stylesheet", href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/styles/atom-one-light.min.css", id="hljs-light"),
        Link(rel="stylesheet", href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/styles/atom-one-dark.min.css", id="hljs-dark"),
        Script(src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/highlight.min.js")
    )

In [ ]:
#| export
def CodeHighlightThemeScript(
    custom_kws:str="" # A space separated string of variable names to highlight
):
    return (
        Script(f"""
            function updateHljsTheme() {{
                const isDark = document.documentElement.classList.contains('dark');
                document.getElementById('hljs-light').disabled = isDark;
                document.getElementById('hljs-dark').disabled = !isDark;
            }}
            updateHljsTheme();

            hljs.registerLanguage('python-custom', function(hljs) {{
                var python = hljs.getLanguage('python');
                python.keywords.built_in += ' {custom_kws}';
                return python;
            }});

            document.addEventListener('basecoat:theme', () => setTimeout(updateHljsTheme, 0));
            document.body.addEventListener('htmx:afterSettle', () => {{
                hljs.highlightAll();
            }});
            hljs.highlightAll();
        """),
    )

### Fasthtml

In [ ]:
#| export
@delegates(fh.FastHTML, keep=True, but=["pico"])
def FastHTML(hdrs=None, ftrs=None, icons=[], code_highlight=True, custom_kws="", offline=False, default_hdrs=True, **kwargs):
    hdrs = make_hdrs(offline) + (hdrs or ()) + (spritesheet,)
    if offline: default_hdrs = False
    ftrs = ftrs or ()
    if code_highlight:
        hdrs += code_highlight_headers()
        ftrs += CodeHighlightThemeScript(custom_kws)
    spritesheet.nms.update(basecamp_icons())
    spritesheet.nms.update(icons)
    return fh.FastHTML(hdrs=hdrs, ftrs=ftrs, pico=False, default_hdrs=default_hdrs, **kwargs)

In [ ]:
app = FastHTML(session_cookie="mysession")
rt = app.route

In [ ]:
#| export
def get_preview(app=None): 
    if not app: app = FastHTML(session_cookie="mysession")
    return partial(HTMX, app=app, host=None, port=None)
p = get_preview()

In [ ]:
c = Div(
    Button('Hey there', cls='btn-outline'),
)
p(c)

In [ ]:
#| export
def slugify(s):
    return re.sub(r"[&/\s]+", "-", s).strip("-").lower()

In [ ]:
slugify("This content")

'this-content'

In [ ]:
#| export
# To easily preview items in a larger container
def Window(*args, cls="h-96"):
    return Div(*args, cls=f"w-full flex flex-col items-center justify-center {cls}")

In [ ]:
#| export
def pw(*args, **kwargs):
    return p(Div(Window(*args, **kwargs), cls="h-100 w-full flex flex-col justify-center items-center"))

In [ ]:
pw(c)

In [ ]:
#| export
class VEnum(Enum):
    def __str__(self): return self.value
    def __add__(self, b): return f"{self.value} {b}"
    def __radd__(self, a): return f"{a} {self.value}"

In [ ]:
class TestEnum(VEnum):
    test1 = "testing-one"
    test2 = "testing-two"

In [ ]:
TestEnum.test1 + TestEnum.test2

'testing-one testing-two'

In [ ]:
"h2" + TestEnum.test1

'h2 testing-one'

### Icons
For quick development, it is easiest to use standalone icons without a spritesheet. Once you are happy with the set of icons you need, you can switch to the spritesheet for efficiency purposes.

In [ ]:
#| export
def Icon(nm, sz=24, vbox=24, stroke=None, stroke_width=None, fill=None, cls=""):
    sym = [ft(t, **attrs) for t,attrs in spritesheet.icons[nm]]
    style = _style_str(stroke, fill, stroke_width)
    return Svg(*sym, cls=f"lucide-icon {cls}", style=style, **sz_attrs(sz, vbox))

In [ ]:
p(Icon("apple"))